# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.
## https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?view=azureml-api-1&tabs=python

In [1]:
from azureml.core import Workspace, Experiment, Environment
from azureml.core.conda_dependencies import CondaDependencies

In [34]:
from azureml.core import Workspace, Experiment, Environment
from azureml.core.conda_dependencies import CondaDependencies

# Connect to the workspace
ws = Workspace.from_config()

# Choose a name for the experiment
experiment_name = 'student-math-experiment-automl'
experiment = Experiment(ws, experiment_name)

# Setting up the environment
env_name = "ML-env"
env = Environment(name=env_name)

# Creating dependencies for the environment
deps = CondaDependencies.create(
    conda_packages=['python=3.9'],
    pip_packages=[
        'azureml-defaults', 
        'azureml-train-automl-runtime',
        'joblib',
        'pandas',
        'numpy',
        'scikit-learn==1.5.1',
        'logging',
        'json',
        'os',
        'azureml-core'
    ]
)

env.python.conda_dependencies = deps

# Register the environment with AzureML
env.register(workspace=ws)

# Prepare the list of installed packages
installed_packages = list(deps.pip_packages) + list(deps.conda_packages)

# Print a professional summary
print(f"Environment '{env_name}' registered successfully.")
print("\nThe following packages have been successfully added to the environment:")
print("-" * 60)
for package in installed_packages:
    print(f"  - {package}")
print("-" * 60)
print("\nThis environment is now ready for use in your experiment.")

Environment 'ML-env' registered successfully.

The following packages have been successfully added to the environment:
------------------------------------------------------------
  - azureml-defaults~=1.57.0
  - azureml-train-automl-runtime~=1.57.0
  - joblib
  - pandas
  - numpy
  - scikit-learn==1.5.1
  - logging
  - json
  - os
  - azureml-core~=1.57.0
  - python=3.9
------------------------------------------------------------

This environment is now ready for use in your experiment.


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
from azureml.core import Dataset

# Load dataset from Azure ML workspace
df = Dataset.get_by_name(workspace=ws, name="student-math")



## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 15,  
    "primary_metric": "normalized_mean_absolute_error",  
    "n_cross_validations": 5, 
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task="regression",  
    training_data=df,  
    label_column_name="G3", 
    compute_target="notebook-compute", 
    enable_early_stopping=True,
    featurization="auto",  
    **automl_settings
) 

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)
remote_run.wait_for_completion(show_output=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
student-math-experiment-automl,AutoML_64c27740-a687-4d21-960d-8cb5b2496dbc,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


Experiment,Id,Type,Status,Details Page,Docs Page
student-math-experiment-automl,AutoML_64c27740-a687-4d21-960d-8cb5b2496dbc,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https:/

{'runId': 'AutoML_64c27740-a687-4d21-960d-8cb5b2496dbc',
 'target': 'notebook-compute',
 'status': 'Completed',
 'startTimeUtc': '2025-03-25T11:30:33.33758Z',
 'endTimeUtc': '2025-03-25T11:52:52.559899Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'normalized_mean_absolute_error',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'notebook-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"student-math-experiment-automl","subscription_id":"48778e11-0fc7-4fc8-a16c-304a430e61a4","resource_group":"student-performance-rg","workspace_name":"student-performance-ws","region":"no

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.
from azureml.widgets import RunDetails

# Show run details in the notebook
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
best_run, best_model = remote_run.get_output()

In [9]:
import joblib

#TODO: Save the best model
print("Best Run ID:", best_run.id)
print("Best Model Properties:")
for key, value in best_run.get_metrics().items():
    print(f"{key}: {value}")


model_filename = "best_automl_model.pkl"
joblib.dump(best_model, model_filename)

Best Run ID: AutoML_64c27740-a687-4d21-960d-8cb5b2496dbc_30
Best Model Properties:
explained_variance: 0.8266964049764752
normalized_mean_absolute_error: 0.06062634943812599
normalized_median_absolute_error: 0.037194053785932116
mean_absolute_error: 1.21252698876252
spearman_correlation: 0.9413303229578883
root_mean_squared_error: 1.8636669312069258
normalized_root_mean_squared_error: 0.0931833465603463
r2_score: 0.8256823757882106
root_mean_squared_log_error: 0.4595906582209502
mean_absolute_percentage_error: 9.635062166855665
median_absolute_error: 0.7438810757186424
normalized_root_mean_squared_log_error: 0.15095656794193785
residuals: aml://artifactId/ExperimentRun/dcid.AutoML_64c27740-a687-4d21-960d-8cb5b2496dbc_30/residuals
predicted_true: aml://artifactId/ExperimentRun/dcid.AutoML_64c27740-a687-4d21-960d-8cb5b2496dbc_30/predicted_true


['best_automl_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [24]:
from azureml.core.model import Model
import os

# Ensure the model file exists
model_file = "best_automl_model.pkl"
if not os.path.isfile(model_file):
    raise FileNotFoundError(f"The model file {model_file} does not exist.")

# Register the model
model = Model.register(
    workspace=ws,
    model_name="student-performance-model",
    model_path=model_file,
    description="AutoML trained model for predicting student performance",
    tags={
        "type": "AutoML",
        "task": "regression", 
        "framework": "scikit-learn"
    }
)

print(f"Model registered: Name={model.name}, ID={model.id}, Version={model.version}")


Registering model student-performance-model
Model registered: Name=student-performance-model, ID=student-performance-model:3, Version=3


In [35]:
from azureml.core.model import Model, InferenceConfig
from azureml.core.webservice import AciWebservice

service_name = 'student-performance-predictor'

# Set up inference configuration
inference_config = InferenceConfig(entry_script='score.py', environment=env)

# ACI configuration (Azure-based deployment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploy the model to ACI (Azure-based deployment)
service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],  
    inference_config=inference_config,
    deployment_config=aci_config,
    overwrite=True
)

# Wait for the deployment to finish
service.wait_for_deployment(show_output=True)

print(f"Scoring URI: {service.scoring_uri}")


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-03-25 15:07:51+00:00 Creating Container Registry if not exists.
2025-03-25 15:07:52+00:00 Building image.

ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 84bcd444-a111-47d8-968a-78258bc6e941
More information can be found here: 
Error:
{
  "code": "EnvironmentBuildFailed",
  "statusCode": 400,
  "message": "Failed building the Environment. You can try debugging locally first. Please refer to https://aka.ms/debugimage#debug-locally for more information."
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 84bcd444-a111-47d8-968a-78258bc6e941
More information can be found here: 
Error:
{
  "code": "EnvironmentBuildFailed",
  "statusCode": 400,
  "message": "Failed building the Environment. You can try debugging locally first. Please refer to https://aka.ms/debugimage#debug-locally for more information."
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Failed\nOperation ID: 84bcd444-a111-47d8-968a-78258bc6e941\nMore information can be found here: \nError:\n{\n  \"code\": \"EnvironmentBuildFailed\",\n  \"statusCode\": 400,\n  \"message\": \"Failed building the Environment. You can try debugging locally first. Please refer to https://aka.ms/debugimage#debug-locally for more information.\"\n}"
    }
}

TODO: In the cell below, send a request to the web service you deployed to test it.

In [30]:
input_data = {
    "data": [
        {
            "school": "GP", "sex": "F", "age": 16, "address": "R",
            "famsize": "LE3", "Pstatus": "T", "Medu": 2, "Fedu": 3,
            "Mjob": "other", "Fjob": "teacher", "reason": "home",
            "guardian": "father", "traveltime": 3, "studytime": 3,
            "failures": 2, "schoolsup": "yes", "famsup": "yes",
            "paid": "yes", "activities": "no", "nursery": "yes",
            "higher": "no", "internet": "yes", "romantic": "no",
            "famrel": 1, "freetime": 2, "goout": 4, "Dalc": 5,
            "Walc": 5, "health": 1, "absences": 8, "G1": 15, "G2": 20
        },
        {
            "school": "GP", "sex": "M", "age": 16, "address": "U",
            "famsize": "GT3", "Pstatus": "A", "Medu": 2, "Fedu": 0,
            "Mjob": "health", "Fjob": "health", "reason": "home",
            "guardian": "other", "traveltime": 3, "studytime": 2,
            "failures": 1, "schoolsup": "no", "famsup": "yes",
            "paid": "yes", "activities": "yes", "nursery": "no",
            "higher": "no", "internet": "yes", "romantic": "no",
            "famrel": 1, "freetime": 4, "goout": 4, "Dalc": 1,
            "Walc": 2, "health": 1, "absences": 18, "G1": 8, "G2": 10
        },
        {
            "school": "GP", "sex": "M", "age": 20, "address": "U",
            "famsize": "LE3", "Pstatus": "T", "Medu": 2, "Fedu": 0,
            "Mjob": "teacher", "Fjob": "other", "reason": "reputation",
            "guardian": "other", "traveltime": 3, "studytime": 1,
            "failures": 0, "schoolsup": "no", "famsup": "yes",
            "paid": "yes", "activities": "no", "nursery": "yes",
            "higher": "yes", "internet": "no", "romantic": "yes",
            "famrel": 1, "freetime": 2, "goout": 2, "Dalc": 3,
            "Walc": 2, "health": 1, "absences": 11, "G1": 17, "G2": 13
        }
    ]
}

In [31]:
import requests
import json

# Replace with your actual endpoint URL
endpoint_url = "https://your-endpoint-url.azurewebsites.net/score"

# If your service requires authentication, include the key
api_key = "your-api-key-if-required"  # Replace with actual key if needed
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}" if api_key else None
}

# Send request to the web service
response = requests.post(endpoint_url, data=json.dumps(input_data), headers=headers)

# Print response
print("Prediction Response:", response.json())

ConnectionError: HTTPSConnectionPool(host='your-endpoint-url.azurewebsites.net', port=443): Max retries exceeded with url: /score (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7efade340190>: Failed to establish a new connection: [Errno -2] Name or service not known'))

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
from azureml.core.webservice import Webservice

# Print logs
print("Service logs:")
print(service.get_logs())

# Delete the service
service.delete()
print(f"Service '{service_name}' has been deleted.")

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
